In [1]:
%pylab inline
from ipyparallel import Client, error
cluster=Client(profile="mpi") 
view=cluster[:]
view.block=True

try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om

Populating the interactive namespace from numpy and matplotlib


# Distributed Components

At times when you need to perform a computation using large input arrays, you may want to perform that computation in multiple processes, where each process operates on some subset of the input values. This may be done purely for performance reasons, or it may be necessary because the entire input will not fit in the memory of a single machine. In any case, this can be accomplished in OpenMDAO using a distributed component. A distributed component is a component that operates on distributed variables. A variable is distributed if each process contains only a part of the whole variable.

We’ve already seen that by using :ref:`src_indices <connect_src_indices>` we can connect an input to only a subset of an output variable. By giving different values for src_indices in each MPI process, we can distribute computations on a distributed output across the processes.

You tell the framework that a Component is a distributed component by setting its `distributed` option to True:

## Component Options

In [2]:
om.show_options_table("openmdao.core.component.Component")

Option,Default,Acceptable Values,Acceptable Types,Description
assembled_jac_type,csc,"['csc', 'dense']",N/A,"Linear solver(s) in this group, if using an assembled jacobian, will use this type."
distributed,False,"[True, False]",['bool'],True if the component has variables that are distributed across multiple processes.


```{note}
If a Component is distributed then all of its outputs are distributed.
```

## Distributed Component Example

The following example shows how to create a distributed component, *DistribComp*, that distributes its computation evenly across the available processes. A second component, *Summer*, sums the values from the distributed component into a scalar output value.

These components can found in the OpenMDAO test suite:

In [2]:
class DistribComp(om.ExplicitComponent):
    """Simple Distributed Component."""

    def initialize(self):
        self.options['distributed'] = True

        self.options.declare('size', types=int, default=1,
                             desc="Size of input and output vectors.")

    def setup(self):
        comm = self.comm
        rank = comm.rank

        size = self.options['size']

        # if comm.size is 2 and size is 15, this results in
        # 8 entries for proc 0 and 7 entries for proc 1
        sizes, offsets = evenly_distrib_idxs(comm.size, size)
        mysize = sizes[rank]
        start = offsets[rank]
        end = start + mysize

        self.add_input('invec', np.ones(mysize, float),
                       src_indices=np.arange(start, end, dtype=int))

        self.add_output('outvec', np.ones(mysize, float))

    def compute(self, inputs, outputs):
        if self.comm.rank == 0:
            outputs['outvec'] = inputs['invec'] * 2.0
        else:
            outputs['outvec'] = inputs['invec'] * -3.0

```{note}
In this example component, we have explicitly specified *src_indices* when adding the input. This is not really necessary in this case, because it replicates the default behavior. If no *src_indices* are specified, OpenMDAO will assume an offset that is the sum of the sizes in all ranks up to the current rank and a range equal to the specified size (the size is given per the usual arguments to `add_input`).
```

In [3]:
class Summer(om.ExplicitComponent):
    """Sums an input array."""

    def initialize(self):
        self.options.declare('size', types=int, default=1,
                             desc="Size of input and output vectors.")

    def setup(self):
        self.add_input('invec', np.ones(self.options['size'], float))

        self.add_output('sum', 0.0, shape=1)

    def compute(self, inputs, outputs):
        outputs['sum'] = np.sum(inputs['invec'])

```{note}
A component that takes a distributed output as input does not need to do anything special as OpenMDAO performs the required MPI operations to make the full value available.
```
This example is run with two processes and a size of 15:

In [4]:
%%px

import numpy as np
import openmdao.api as om
from openmdao.test_suite.components.distributed_components import DistribComp, Summer

size = 15

model = om.Group()

# Distributed component "C2" requires an IndepVarComp to supply inputs.
model.add_subsystem("indep", om.IndepVarComp('x', np.zeros(size)))
model.add_subsystem("C2", DistribComp(size=size))
model.add_subsystem("C3", Summer(size=size))

model.connect('indep.x', 'C2.invec')
model.connect('C2.outvec', 'C3.invec')

prob = om.Problem(model)
prob.setup()

[stderr:0] /home/kmoore/Work/OpenMDAO/OpenMDAO/openmdao/core/component.py:522: DeprecationWarning:'C2' <class DistribComp>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.
[stderr:1] /home/kmoore/Work/OpenMDAO/OpenMDAO/openmdao/core/component.py:522: DeprecationWarning:'C2' <class DistribComp>: Passing `src_indices` as an arg to `add_input` isdeprecated and will become an error in a future release.  Add `src_indices` to a `promotes` or `connect` call instead.


Out[0:8]: <openmdao.core.problem.Problem at 0x7fdb16ac1eb0>

Out[1:8]: <openmdao.core.problem.Problem at 0x7f4f5599fd00>

In [5]:
%%px

prob.set_val('indep.x', np.ones(size))
prob.run_model()

print(prob.get_val('C2.invec'))

[stdout:0] [1. 1. 1. 1. 1. 1. 1. 1.]
[stdout:1] [1. 1. 1. 1. 1. 1. 1.]


In [6]:
%%px

print(prob.get_val('C2.outvec'))

[stdout:0] [2. 2. 2. 2. 2. 2. 2. 2.]
[stdout:1] [-3. -3. -3. -3. -3. -3. -3.]


In [7]:
%%px

print(prob.get_val('C3.sum'))

[stdout:0] [-5.]
[stdout:1] [-5.]


In [8]:
%%px

from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('C2.invec'),
                  np.ones((8,)) if model.comm.rank == 0 else np.ones((7,)))
assert_near_equal(prob.get_val('C2.outvec'),
                  2*np.ones((8,)) if model.comm.rank == 0 else -3*np.ones((7,)))
assert_near_equal(prob.get_val('C3.sum'), -5.)

Out[0:12]: 0.0

Out[1:12]: 0.0

```{note}
In this example, we introduce a new component called an :ref:`IndepVarComp<comp-type-1-indepvarcomp>`. If you used OpenMDAO prior to version 3.2, then you are familiar with this component. It is used to define an independent variable.

You usually do not have to define these because OpenMDAO defines and uses them automatically for all unconnected inputs in your model. However, when we define a distributed input, we often use the “src_indices” attribute to determine the allocation of that input to the processors that the component sees. For some sets of these indices, it isn’t possible to easily determine the full size of the corresponding independent variable, and the *IndepVarComp* cannot be created automatically. So, for unconnected inputs on a distributed component, you must manually create one, as we did in this example.
```

## Distributed Component with Derivatives

Derivatives can be computed for distributed components as shown in the following variation on the example. Also, in this version, we have taken advantage of the automatic determination of *src_indices*.

In [9]:
class DistribCompDerivs(om.ExplicitComponent):
    """Simple Distributed Component with Derivatives."""

    def initialize(self):
        self.options['distributed'] = True

        self.options.declare('size', types=int, default=1,
                             desc="Size of input and output vectors.")

    def setup(self):
        comm = self.comm
        rank = comm.rank

        size = self.options['size']

        # if comm.size is 2 and size is 15, this results in
        # 8 entries for proc 0 and 7 entries for proc 1
        sizes, _ = evenly_distrib_idxs(comm.size, size)
        self.mysize = mysize = sizes[rank]

        # don't set src_indices on the input, just use default behavior
        self.add_input('invec', np.ones(mysize, float))
        self.add_output('outvec', np.ones(mysize, float))

    def setup_partials(self):
        # declare partial derivatives (diagonal of mysize)
        self.declare_partials('outvec', 'invec',
                              rows=np.arange(0, self.mysize),
                              cols=np.arange(0, self.mysize))

    def compute(self, inputs, outputs):
        if self.comm.rank == 0:
            outputs['outvec'] = inputs['invec'] * 2.0
        else:
            outputs['outvec'] = inputs['invec'] * -3.0

    def compute_partials(self, inputs, J):
        # get mysize from the input vector for this process
        mysize = inputs['invec'].size

        if self.comm.rank == 0:
            J['outvec', 'invec'] = np.ones((mysize,)) * 2.0
        else:
            J['outvec', 'invec'] = np.ones((mysize,)) * -3.0

In [10]:
class SummerDerivs(om.ExplicitComponent):
    """Sums an input array."""

    def initialize(self):
        self.options.declare('size', types=int, default=1,
                             desc="Size of input and output vectors.")

    def setup(self):
        self.add_input('invec', np.ones(self.options['size'], float))

        self.add_output('sum', 0.0, shape=1)

    def setup_partials(self):
        # the derivative is constant
        self.declare_partials('sum', 'invec', val=1.)

    def compute(self, inputs, outputs):
        outputs['sum'] = np.sum(inputs['invec'])

This example is again run with two processes and a `size` of 15. We can use :ref:`assert_check_partials<feature_unit_testing_partials>` to verify that the partial derivatives are calculated correctly.

In [11]:
%%px

import numpy as np
import openmdao.api as om
from openmdao.test_suite.components.distributed_components import DistribCompDerivs, SummerDerivs
from openmdao.utils.assert_utils import assert_check_partials

size = 15

model = om.Group()

# Distributed component "C2" requires an IndepVarComp to supply inputs.
model.add_subsystem("indep", om.IndepVarComp('x', np.zeros(size)))
model.add_subsystem("C2", DistribCompDerivs(size=size))
model.add_subsystem("C3", SummerDerivs(size=size))

model.connect('indep.x', 'C2.invec')
model.connect('C2.outvec', 'C3.invec')

prob = om.Problem(model)
prob.setup()

[stderr:0] /home/kmoore/Work/OpenMDAO/OpenMDAO/openmdao/core/component.py:913: UserWarning:'C2' <class DistribCompDerivs>: Component is distributed but input 'C2.invec' was added without src_indices. Setting src_indices to np.arange(0, 8, dtype=INT_DTYPE).reshape((8,)).
[stderr:1] /home/kmoore/Work/OpenMDAO/OpenMDAO/openmdao/core/component.py:913: UserWarning:'C2' <class DistribCompDerivs>: Component is distributed but input 'C2.invec' was added without src_indices. Setting src_indices to np.arange(8, 15, dtype=INT_DTYPE).reshape((7,)).


Out[0:13]: <openmdao.core.problem.Problem at 0x7fdb169ac5e0>

Out[1:13]: <openmdao.core.problem.Problem at 0x7f4f559832b0>

In [12]:
%%px

prob.set_val('indep.x', np.ones(size))
prob.run_model()

print(prob.get_val('C2.invec'))

[stdout:0] [1. 1. 1. 1. 1. 1. 1. 1.]
[stdout:1] [1. 1. 1. 1. 1. 1. 1.]


In [13]:
%%px

print(prob.get_val('C2.outvec'))

[stdout:0] [2. 2. 2. 2. 2. 2. 2. 2.]
[stdout:1] [-3. -3. -3. -3. -3. -3. -3.]


In [14]:
%%px

print(prob.get_val('C3.sum'))

[stdout:0] [-5.]
[stdout:1] [-5.]


In [15]:
%%px

assert_check_partials(prob.check_partials())

[stdout:0] 
---------------------------------
Component: DistribCompDerivs 'C2'
---------------------------------
  C2: 'outvec' wrt 'invec'
    Analytic Magnitude : 5.656854e+00
          Fd Magnitude : 5.656854e+00 (fd:forward)
    Absolute Error (Jan - Jfd) : 4.653704e-10

    Relative Error (Jan - Jfd) / Jfd : 8.226664e-11
    MPI Rank 0

    Raw Analytic Derivative (Jfor)
[[2. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2.]]

    Raw FD Derivative (Jfd)
[[2. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 2. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0.]
 [0. 0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 0. 0. 2.]]

 - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
----------------------------
Component: SummerDerivs 'C3'
----------------------------
  C3: 's

In [16]:
%%px

J = prob.compute_totals(of=['C2.outvec'], wrt=['indep.x'])
print(J[('C2.outvec', 'indep.x')])

[stdout:0] 
[[ 2. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0.  2. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0.  2. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0.  2. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0.  2. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0.  2. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0.  2. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0.  2. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -3. -0. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -3. -0. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -3. -0. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -3. -0. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -3. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -3. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -3.]]
[stdout:1] 
[[ 2. -0. -0. -0. -0. -0. -0. 

In [19]:
%%px

assert_near_equal(J[('C2.outvec', 'indep.x')],
                  np.eye(15)*np.append(2*np.ones(8), -3*np.ones(7)))

Out[0:32]: 0.0

Out[1:32]: 0.0